<a href="https://colab.research.google.com/github/Re14m/training/blob/master/2022-0603_recipie226.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [scikit-surpriseを使った映画のレコメンデーションレシピ](https://axross-recipe.com/recipes/226)

## 環境準備

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ライブラリのインストール
!pip install jupyter
!pip install pandas
!pip install scikit-surprise

## データセットの準備

In [ ]:
# datasetのダウンロード
!wget "https://files.grouplens.org/datasets/movielens/ml-100k.zip" /content/

In [ ]:
# datasetの解凍
!unzip "ml-100k.zip"

In [ ]:
# datasetの確認
!ls ml-100k

## データセットの準備

In [ ]:
# ライブラリのインポート
import pandas as pd
from datetime import datetime

# データの読込
df_data = pd.read_csv("ml-100k/u.data",sep="\t", header=None,
                      names=["user id", "movie id", "rating", "timestamp"], 
                      parse_dates=[3], date_parser=lambda x: datetime.fromtimestamp(int(x)))
df_item = pd.read_csv("ml-100k/u.item",sep="|", encoding = "ISO-8859-1", header=None, 
                      names= ["movie id", "movie title", "release date"],
                      usecols=[0,1,2], parse_dates=[2], index_col=0)

In [ ]:
# データの表示
df_data.head()

In [ ]:
# データの表示
df_item.head()

## データの加工

In [ ]:
# ライブラリのインポート
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

# dataset加工
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(df_data[["user id", "movie id", "rating"]], reader)

## SVDの学習

In [ ]:
from surprise import SVD
model = SVD(random_state=1)

model.fit(data.build_full_trainset())

## SVDの予測

In [ ]:
# 一人が見た映画の数の最小値
aggregated = df_data[["user id", "movie id"]].groupby("user id").count().rename(columns={"movie id":"count"})
aggregated.min()

In [ ]:
# 鑑賞数が20になった人を抽出
aggregated[ aggregated["count"]==20 ]

In [ ]:
# サンプルとしてユーザIDが364のタイトル一覧を取得
uid = 364
df_data[df_data["user id"] == uid].merge(df_item,"left", on="movie id")

In [ ]:
# ユーザID 364が映画ID 1 "ToyStory"をどう評価するかを予測する
iid = 1
model.predict(uid=uid, iid=iid).est

In [ ]:
# 関数の定義（与えられたユーザと候補映画リストについて、それぞれの映画の評価予測を計算し降順にソート）
def get_ranking(user_id, model, candidates):
    pred = [(i, model.predict(user_id, i).est) for i in candidates]
    return sorted(pred, key=lambda x: x[1], reverse=True)

In [ ]:
# ユーザID 364のおすすめリストを予測
watched = set(df_data[df_data["user id"] == uid]["movie id"])
r = get_ranking(uid, model, set(df_item.index) - watched)

In [ ]:
# おすすめリストを取得
r[:30]

In [ ]:
# 関数の定義（値をタイトルとして表示させる）
def show_result(res):
    for movie, score in res:
        print("{:4d} {:70s} {:f}".format(movie, df_item.loc[movie]["movie title"], score))

In [ ]:
# 最後おすすめリストを表示
show_result(r[:30])

## 映画のレコメンデーション

In [ ]:
# 人工的にユーザを追加する
df_data["user id"].max()

In [ ]:
# "StarWars"で一覧を検索する
df_item[df_item["movie title"].str.contains("Star Wars")]

In [ ]:
# ユーザIDが944の人を人工的に加えて、その人は"Star Wars"（映画ID=50）を5と評価したもの
uid = 944
iid = 50
df_data2 = df_data.append({"user id":uid, "movie id": iid, "rating": 5}, ignore_index=True).convert_dtypes()

In [ ]:
# データが加えられていることを確認
df_data2.tail()

In [ ]:
# 実際におすすめを表示する
model = SVD(random_state=1)
data2 = Dataset.load_from_df(df_data2[["user id", "movie id", "rating"]], reader)
model.fit(data2.build_full_trainset())

r = get_ranking(uid, model, [x for x in df_item.index if x!=iid])

show_result(r[:30])

## アルゴリズムの評価

In [ ]:
# データの分割
from surprise.model_selection import train_test_split
from surprise import accuracy
trainset, testset = train_test_split(data, test_size=0.2)

In [ ]:
# KNNBasic, NMF, SVDを評価して比較
from surprise.prediction_algorithms.knns import KNNBasic
from surprise.prediction_algorithms.matrix_factorization import NMF
algorithms = [KNNBasic, NMF, SVD]
algo_names = ["KNNBasic", "NMF", "SVD"]
for algo, name in zip(algorithms, algo_names):
    model = algo()
    model.fit(trainset)
    predictions = model.test(testset)
    print(name)
    print(accuracy.rmse(predictions,verbose=False))